In [1]:
import os
# Find the latest version of spark 2.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.0'
spark_version = 'spark-3.0.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Fetched 252 kB in 2s (109 kB/s)

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [3]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Pet_Products_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   28794885| REAKC26P07MDN|B00Q0K9604|     510387886|(8-Pack) EZwhelp ...|    Pet Products|          5|            0|          0|   N|                Y|A great purchase ...|Best belly bands ...| 2015-08-31|
|         US|   11488901|R3NU7OMZ4HQIEG|B00MBW5O9W|     912374672|Warren Eckstein's...|    Pet Products|          2|    

In [4]:
# Create the vine_table. DataFrame
vine_df = df.select(['review_id', 'star_rating', 'helpful_votes', 'total_votes', 'vine', 'verified_purchase'])
vine_df.show(5)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| REAKC26P07MDN|          5|            0|          0|   N|                Y|
|R3NU7OMZ4HQIEG|          2|            0|          1|   N|                Y|
|R14QJW3XF8QO1P|          5|            0|          0|   N|                Y|
|R2HB7AX0394ZGY|          5|            0|          0|   N|                Y|
| RGKMPDQGSAHR3|          5|            0|          0|   N|                Y|
+--------------+-----------+-------------+-----------+----+-----------------+
only showing top 5 rows



In [5]:
filtered_df = vine_df.filter('total_votes>=20')
filtered_df.show(5)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R21KC552Y6HL8X|          1|           27|         31|   N|                Y|
| RX9WC9FTIR1XR|          5|           25|         25|   N|                Y|
| RGDCOU1KBHMNG|          3|           29|         31|   N|                Y|
| RVTYWID2TPMMY|          2|           35|         42|   N|                Y|
|R2CMPZ5VESGRLY|          4|           27|         28|   N|                Y|
+--------------+-----------+-------------+-----------+----+-----------------+
only showing top 5 rows



In [6]:
from pyspark.sql.functions import col
filtered_df.withColumn('helpful_votes', col('helpful_votes').cast('Integer'))
filtered_df.withColumn('total_votes', col('total_votes').cast('Integer'))

DataFrame[review_id: string, star_rating: int, helpful_votes: int, total_votes: int, vine: string, verified_purchase: string]

In [7]:
new_df = filtered_df.withColumn('vote_ratio', col('helpful_votes').cast('Integer') / col('total_votes').cast('Integer'))
new_df.show(5)

+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|        vote_ratio|
+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|R21KC552Y6HL8X|          1|           27|         31|   N|                Y|0.8709677419354839|
| RX9WC9FTIR1XR|          5|           25|         25|   N|                Y|               1.0|
| RGDCOU1KBHMNG|          3|           29|         31|   N|                Y|0.9354838709677419|
| RVTYWID2TPMMY|          2|           35|         42|   N|                Y|0.8333333333333334|
|R2CMPZ5VESGRLY|          4|           27|         28|   N|                Y|0.9642857142857143|
+--------------+-----------+-------------+-----------+----+-----------------+------------------+
only showing top 5 rows



In [8]:
new_filtered_df = new_df.filter('vote_ratio>=0.5')
new_filtered_df.show(5)

+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|        vote_ratio|
+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|R21KC552Y6HL8X|          1|           27|         31|   N|                Y|0.8709677419354839|
| RX9WC9FTIR1XR|          5|           25|         25|   N|                Y|               1.0|
| RGDCOU1KBHMNG|          3|           29|         31|   N|                Y|0.9354838709677419|
| RVTYWID2TPMMY|          2|           35|         42|   N|                Y|0.8333333333333334|
|R2CMPZ5VESGRLY|          4|           27|         28|   N|                Y|0.9642857142857143|
+--------------+-----------+-------------+-----------+----+-----------------+------------------+
only showing top 5 rows



In [9]:
vine_paid = new_filtered_df.filter(new_filtered_df.vine == 'Y')
vine_paid.show(5)

+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|        vote_ratio|
+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|R3A71VR1JZD8WF|          2|           27|         30|   Y|                N|               0.9|
|R16OMUJIGI18JZ|          5|           72|         72|   Y|                N|               1.0|
|R3TS8ZP2FHQ9XR|          5|           39|         42|   Y|                N|0.9285714285714286|
|R2MHP919VZN7DI|          5|           29|         30|   Y|                N|0.9666666666666667|
| RD2BCTVS59A5L|          2|           20|         20|   Y|                N|               1.0|
+--------------+-----------+-------------+-----------+----+-----------------+------------------+
only showing top 5 rows



In [10]:
vine_unpaid = new_filtered_df.filter(new_filtered_df.vine == 'N')
vine_unpaid.show(5)

+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|        vote_ratio|
+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|R21KC552Y6HL8X|          1|           27|         31|   N|                Y|0.8709677419354839|
| RX9WC9FTIR1XR|          5|           25|         25|   N|                Y|               1.0|
| RGDCOU1KBHMNG|          3|           29|         31|   N|                Y|0.9354838709677419|
| RVTYWID2TPMMY|          2|           35|         42|   N|                Y|0.8333333333333334|
|R2CMPZ5VESGRLY|          4|           27|         28|   N|                Y|0.9642857142857143|
+--------------+-----------+-------------+-----------+----+-----------------+------------------+
only showing top 5 rows



In [11]:
paid_total = vine_paid.count()
paid_total

170

In [12]:
paid_fivestar_df = vine_paid.filter(vine_paid.star_rating == 5)
paid_fivestar_count = paid_fivestar_df.count()
paid_fivestar_count

65

In [13]:
paid_fivestar_percent = paid_fivestar_count / paid_total
paid_fivestar_percent

0.38235294117647056

In [14]:
unpaid_total = vine_unpaid.count()
unpaid_total

37840

In [15]:
unpaid_fivestar_df = vine_unpaid.filter(vine_unpaid.star_rating == 5)
unpaid_fivestar_count = unpaid_fivestar_df.count()
unpaid_fivestar_count

20612

In [16]:
unpaid_fivestar_percent = unpaid_fivestar_count / unpaid_total
unpaid_fivestar_percent

0.5447145877378435